<a href="https://colab.research.google.com/github/abhisekdey12/python_codes/blob/master/Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install pycaret

     |████████████████████████████████| 266kB 5.0MB/s 
     |████████████████████████████████| 174kB 42.3MB/s 
     |████████████████████████████████| 1.4MB 34.7MB/s 
     |████████████████████████████████| 14.2MB 211kB/s 
     |████████████████████████████████| 112kB 37.2MB/s 
     |████████████████████████████████| 6.8MB 36.0MB/s 
     |████████████████████████████████| 276kB 41.6MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 1.7MB 25.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 256kB 48.7MB/s 
     |████████████████████████████████| 2.0MB 30.9MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 1.1MB 28.8MB/s 
     |████████████████████████████████| 174kB 49.4MB/s

In [4]:
from pycaret.regression import *

In [5]:
import pandas as pd
import numpy as np

In [8]:
from google.colab import files
uploaded = files.upload()

Saving TS_final.csv to TS_final.csv


In [21]:
import io
data = pd.read_csv(io.BytesIO(uploaded['TS_final.csv']))

In [22]:
data['Date'] = pd.to_datetime(data['newdf$TIME'])
data.head()

,newdf$TIME,newdf$MMSI,Date
0,2020-01-01,39,2020-01-01
1,2020-01-02,44,2020-01-02
2,2020-01-03,54,2020-01-03
3,2020-01-04,48,2020-01-04
4,2020-01-05,50,2020-01-05


In [23]:
# extract month and year from dates
data['Week'] = [i.week for i in data['Date']]
data['Month'] = [i.month for i in data['Date']]
data['Year'] = [i.year for i in data['Date']]

# create a sequence of numbers
data['Series'] = np.arange(1,len(data)+1)

# drop unnecessary columns and re-arrange
data.drop(['Date', 'newdf$TIME'], axis=1, inplace=True)
data = data[['Series', 'Year', 'Month', 'Week', 'newdf$MMSI']] 

In [26]:
data.tail(10)

,Series,Year,Month,Week,newdf$MMSI
172,173,2020,6,25,40
173,174,2020,6,26,36
174,175,2020,6,26,29
175,176,2020,6,26,39
176,177,2020,6,26,45
177,178,2020,6,26,44
178,179,2020,6,26,39
179,180,2020,6,26,43
180,181,2020,6,27,54
181,182,2020,6,27,54


In [27]:
data.head(10)

,Series,Year,Month,Week,newdf$MMSI
0,1,2020,1,1,39
1,2,2020,1,1,44
2,3,2020,1,1,54
3,4,2020,1,1,48
4,5,2020,1,1,50
5,6,2020,1,2,46
6,7,2020,1,2,57
7,8,2020,1,2,55
8,9,2020,1,2,42
9,10,2020,1,2,48


In [29]:
# split data into train-test set
train = data[data['Week'] < 25]
test = data[data['Week'] >= 25]

# check shape
train.shape, test.shape

((166, 5), (16, 5))

In [50]:
# import the regression module
from pycaret.regression import *

# initialize setup
s = setup(data = train, test_data = test, target = 'newdf$MMSI', fold_strategy = 'timeseries', numeric_features = ['Year', 'Series','Month'], fold = 3, transform_target = True, remove_multicollinearity= True, session_id = 123)

,Description,Value
0,session_id,123
1,Target,newdf$MMSI
2,Original Data,"(166, 5)"
3,Missing Values,False
4,Numeric Features,4
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(166, 2)"


In [51]:
best = compare_models(sort = 'MAPE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,5.8444,58.3471,7.5269,-0.3188,0.1868,0.1550,0.0233
en,Elastic Net,5.8903,58.8800,7.5661,-0.3390,0.1876,0.1555,0.0200
br,Bayesian Ridge,5.8990,59.1389,7.5812,-0.3430,0.1881,0.1561,0.0233
huber,Huber Regressor,6.0432,61.7407,7.6771,-0.3629,0.1890,0.1599,0.0300
lr,Linear Regression,6.1003,62.2420,7.7559,-0.4097,0.1915,0.1603,0.9533
ridge,Ridge Regression,6.1003,62.2414,7.7559,-0.4097,0.1915,0.1603,0.0200
lar,Least Angle Regression,6.1003,62.2420,7.7559,-0.4097,0.1915,0.1603,0.0233
omp,Orthogonal Matching Pursuit,6.1003,62.2420,7.7559,-0.4097,0.1915,0.1603,0.0233
lightgbm,Light Gradient Boosting Machine,6.0266,62.6007,7.7745,-0.3681,0.1928,0.1647,0.2000
knn,K Neighbors Regressor,6.5519,72.0941,8.2234,-0.5505,0.2009,0.1751,0.0933


In [52]:
prediction_holdout = predict_model(best);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Lasso Regression,5.7509,63.296398,7.9559,-0.33,0.1803,0.1269


In [53]:
# generate predictions on the original dataset
predictions = predict_model(best, data=data)

# generate predictions on train dataset
predictions_train = predict_model(best, data=train)

# generate predictions on test dataset
predictions_test = predict_model(best, data=test)

In [54]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual))

In [56]:
predictions_train.head()

,Series,Year,Month,Week,newdf$MMSI,Label
0,1,2020,1,1,39,47.879929
1,2,2020,1,1,44,47.828705
2,3,2020,1,1,54,47.777451
3,4,2020,1,1,48,47.726173
4,5,2020,1,1,50,47.674866


In [60]:
#Metric Values for train
print('MAE = ',mean_absolute_error(predictions_train['newdf$MMSI'],predictions_train['Label']))
print('RMSE = ',mean_squared_error(predictions_train['newdf$MMSI'],predictions_train['Label'],squared=False))
print('MAPE = ',mape(predictions_train['newdf$MMSI'],predictions_train['Label']))

MAE =  5.074447884617082
RMSE =  6.548402748529934
MAPE =  0.12824370387356748


In [61]:
#Metric values for test
print('MAE = ',mean_absolute_error(predictions_test['newdf$MMSI'],predictions_test['Label']))
print('RMSE = ',mean_squared_error(predictions_test['newdf$MMSI'],predictions_test['Label'],squared=False))
print('MAPE = ',mape(predictions_test['newdf$MMSI'],predictions_test['Label']))

MAE =  5.750870704650879
RMSE =  7.955903777508448
MAPE =  0.1268691796018617


In [ ]:
# add a date column in the dataset
predictions['Date'] = pd.date_range(start='2020-01-01', end = '2020-06-30', freq = 'D')

In [48]:
predictions.tail()

,Series,Year,Month,Week,newdf$MMSI,Label,Date
177,178,2020,6,26,44,38.330799,2020-06-26
178,179,2020,6,26,39,38.273655,2020-06-27
179,180,2020,6,26,43,38.216473,2020-06-28
180,181,2020,6,27,54,38.159245,2020-06-29
181,182,2020,6,27,54,38.101974,2020-06-30


In [49]:
predictions.to_csv('output.csv') 
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>